In [1]:
#CELL1
import os, gc
from collections import defaultdict

import pandas as pd
from tqdm.auto import tqdm
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

$$ $$

In [2]:
#CELL2
def read_train_terms(path):
    mapping = defaultdict(list)
    df = pd.read_csv(path, sep="\t", header=None, names=["protein","go","ont"], dtype=str)
    for _, r in tqdm(df.iterrows(), total=len(df)): 
        mapping[r.protein].append(r.go)
    print(f"[io] Read training annotations for {len(mapping)} proteins from {path}")
    return mapping

def parse_obo(go_obo_path):
    parents = defaultdict(set)
    children = defaultdict(set)
    
    if not os.path.exists(go_obo_path): 
        return parents, children
        
    with open(go_obo_path,"r") as f:
        cur_id=None
        for line in f:
            line=line.strip()
            if line=="[Term]": 
                cur_id=None
            elif line.startswith("id: "): 
                cur_id=line.split("id: ")[1].strip()
            elif line.startswith("is_a: "):
                pid=line.split()[1].strip()
                if cur_id: 
                    parents[cur_id].add(pid)
                    children[pid].add(cur_id)
            elif line.startswith("relationship: part_of "):
                parts=line.split(); 
                if len(parts)>=3:
                    pid=parts[2].strip()
                    if cur_id: 
                        parents[cur_id].add(pid)
                        children[pid].add(cur_id)
    print(f"[io] Parsed OBO: {len(parents)} nodes with parents")
    return parents, children

def get_ancestors(go_id, parents):
    ans=set()
    stack=[go_id]
    while stack:
        cur=stack.pop()
        for p in parents.get(cur,[]): 
            if p not in ans:
                ans.add(p)
                stack.append(p)
    return ans

def get_descendants(go_id, children_map):
    desc = set()
    stack = [go_id]
    while stack:
        cur = stack.pop()
        for child in children_map.get(cur, []):
            if child not in desc:
                desc.add(child)
                stack.append(child)
    return desc

def compute_go_depth(go_term, parents_map, cache={}):
    """Compute depth of GO term in hierarchy"""
    if go_term in cache:
        return cache[go_term]
    
    if go_term not in parents_map or len(parents_map[go_term]) == 0:
        cache[go_term] = 0
        return 0
    
    max_depth = 0
    for parent in parents_map[go_term]:
        depth = compute_go_depth(parent, parents_map, cache) + 1
        max_depth = max(max_depth, depth)
    
    cache[go_term] = max_depth
    return max_depth


$$ $$

In [3]:
print("="*80)
print("CAFA-6 STACKING META-LEARNER PIPELINE")
print("="*80)

train_terms = read_train_terms("/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv")
parents_map, children_map = parse_obo("/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo")


CAFA-6 STACKING META-LEARNER PIPELINE


  0%|          | 0/537028 [00:00<?, ?it/s]

[io] Read training annotations for 82405 proteins from /kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv
[io] Parsed OBO: 40121 nodes with parents


$$ $$

In [4]:
#CELL3
print("\n[Computing GO Statistics for Meta-Features]")

# Compute depth for all GO terms
print("Computing GO depths...")
go_depths = {}
all_go_terms = set(parents_map.keys()) | set(children_map.keys())
for go_term in tqdm(all_go_terms, desc="GO depths"):
    go_depths[go_term] = compute_go_depth(go_term, parents_map)

# Compute specificity (number of descendants)
print("Computing GO specificity...")
go_specificity = {}
for go_term in tqdm(all_go_terms, desc="GO specificity"):
    go_specificity[go_term] = len(get_descendants(go_term, children_map))

# Compute GO term frequency in training data
print("Computing GO term frequency...")
go_frequency = defaultdict(int)
for protein, terms in train_terms.items():
    for term in terms:
        go_frequency[term] += 1

total_annotations = sum(go_frequency.values())
go_frequency_norm = {k: v/total_annotations for k, v in go_frequency.items()}

print(f"[✓] GO Statistics computed")
print(f"    - GO terms with depth: {len(go_depths)}")
print(f"    - GO terms with specificity: {len(go_specificity)}")
print(f"    - GO terms with frequency: {len(go_frequency)}")


[Computing GO Statistics for Meta-Features]
Computing GO depths...


GO depths:   0%|          | 0/40125 [00:00<?, ?it/s]

Computing GO specificity...


GO specificity:   0%|          | 0/40125 [00:00<?, ?it/s]

Computing GO term frequency...
[✓] GO Statistics computed
    - GO terms with depth: 40125
    - GO terms with specificity: 40125
    - GO terms with frequency: 26126


In [5]:
go_annotations = pd.read_csv('/kaggle/input/protein-go-annotations/goa_uniprot_all.csv')
go_annotations = go_annotations.drop_duplicates()
print(f'[+] Dataset shape: {go_annotations.shape}')


[+] Dataset shape: (2583077, 3)


In [6]:
print(f"\n[Processing Negative Annotations]")
negative_annots = go_annotations[go_annotations['qualifier'].str.contains('NOT', na=False)]
negative_annots = negative_annots.drop(columns=['qualifier']).drop_duplicates()
negative_annots = negative_annots.groupby('protein_id')['go_term'].apply(list).to_dict()

propagated={}
for p in tqdm(negative_annots.keys(), desc="Propagating negatives"):
    terms=set(negative_annots[p])
    extra=set()
    for t in list(terms): 
        extra |= get_descendants(t, children_map)
    propagated[p] = sorted(terms | extra)
        
negative_annots = propagated

rows = [(protein_id, go_term) for protein_id, terms in negative_annots.items() for go_term in terms]
negative_df = pd.DataFrame(rows, columns=["protein_id", "go_term"])
negative_df['pred_key'] = negative_df.protein_id.apply(str) + '_' + negative_df.go_term.apply(str)
negative_keys = set(negative_df['pred_key'])

del negative_df
gc.collect()

print(f"Total unique negative protein-GO pairs: {len(negative_keys)}")


[Processing Negative Annotations]


Propagating negatives:   0%|          | 0/3863 [00:00<?, ?it/s]

Total unique negative protein-GO pairs: 238553


In [7]:
print(f"\n[Creating Ground Truth]")
go_annotations = pd.read_csv('/kaggle/input/protein-go-annotations/goa_uniprot_all.csv')
go_annotations = go_annotations[~go_annotations['qualifier'].str.contains('NOT', na=False)]
go_annotations.drop(columns=['qualifier'], inplace=True)
go_annotations = go_annotations.drop_duplicates()
go_annotations['score'] = round(1.0, 3)
go_annotations['pred_key'] = go_annotations['protein_id'].astype(str) + '_' + go_annotations['go_term'].astype(str)
go_annotations = go_annotations[~go_annotations['pred_key'].isin(negative_keys)]
goa_pred_keys = set(go_annotations['pred_key'])

print(f"Ground truth pairs: {len(goa_pred_keys)}")



[Creating Ground Truth]
Ground truth pairs: 2462485


$$ $$

In [8]:
def load_submission(path, chunksize=50000, num_rows=None):
    total = None
    if num_rows:
        total = int(num_rows / chunksize) + 1 
    chunks = []
    for chunk in tqdm(pd.read_csv(path, sep='\t', header=None, chunksize=chunksize), total=total):
        chunk['pred_key'] = chunk[0].astype(str) + '_' + chunk[1].astype(str)
        chunks.append(chunk)
    df = pd.concat(chunks, ignore_index=True)
    df.columns = ['protein_id', 'go_term', 'score', 'pred_key']
    return df

In [9]:
print(f"\n[Loading Base Model Predictions]")
print(f"Model A...")
A = load_submission('/kaggle/input/cafa-6-protein-function-starter-eda-model/submission.tsv')

print(f"Model B...")
B = load_submission('/kaggle/input/k/daovanda2405/cafa-6-protein-function-starter-eda-model/submission.tsv')

print(f"A shape: {A.shape}, B shape: {B.shape}")

# Remove ground truth
A = A[~A.pred_key.isin(goa_pred_keys)]
B = B[~B.pred_key.isin(goa_pred_keys)]

# Soft negative filtering
print(f"\n[Soft Negative Filtering]")
A_neg_mask = A['pred_key'].isin(negative_keys)
B_neg_mask = B['pred_key'].isin(negative_keys)
A.loc[A_neg_mask, 'score'] *= 0.1
B.loc[B_neg_mask, 'score'] *= 0.1
A = A[A['score'] >= 0.001]
B = B[B['score'] >= 0.001]

print(f"After filtering - A: {A.shape}, B: {B.shape}")


[Loading Base Model Predictions]
Model A...


0it [00:00, ?it/s]

Model B...


0it [00:00, ?it/s]

A shape: (14090857, 4), B shape: (47706964, 4)

[Soft Negative Filtering]
After filtering - A: (12402396, 4), B: (46669291, 4)


$$ $$

##  🧹 Post-Processing

In [10]:
def create_meta_features(row, go_depths, go_specificity, go_frequency_norm):
    """Extract meta-features from a prediction row"""
    features = {}
    
    # Score features
    features['score_A'] = row.get('score_A', 0)
    features['score_B'] = row.get('score_B', 0)
    features['score_mean'] = (features['score_A'] + features['score_B']) / 2
    features['score_max'] = max(features['score_A'], features['score_B'])
    features['score_min'] = min(features['score_A'], features['score_B'])
    features['score_diff'] = abs(features['score_A'] - features['score_B'])
    features['score_std'] = np.std([features['score_A'], features['score_B']])
    features['score_product'] = features['score_A'] * features['score_B']
    features['score_ratio'] = features['score_max'] / (features['score_min'] + 0.001)
    
    # GO term features
    go_term = row['go_term']
    features['go_depth'] = go_depths.get(go_term, 0)
    features['go_specificity'] = go_specificity.get(go_term, 0)
    features['go_frequency'] = go_frequency_norm.get(go_term, 0)
    
    # Agreement features
    features['models_agree'] = 1 if features['score_diff'] < 0.1 else 0
    features['both_high_conf'] = 1 if (features['score_A'] > 0.7 and features['score_B'] > 0.7) else 0
    
    return features

print(f"\n[Creating Meta-Features Dataset]")

# Merge A and B
A_keys = set(A.pred_key)
B_keys = set(B.pred_key)
intersect_keys = A_keys & B_keys

print(f"Intersection: {len(intersect_keys)} predictions")

# Get intersection
A_inter = A[A.pred_key.isin(intersect_keys)].copy()
B_inter = B[B.pred_key.isin(intersect_keys)].copy()

merged = A_inter.merge(
    B_inter[['pred_key', 'score']],
    on='pred_key',
    suffixes=('_A', '_B')
)

print(f"Creating meta-features for {len(merged)} predictions...")

# Create features
meta_features_list = []
for idx, row in tqdm(merged.iterrows(), total=len(merged), desc="Meta-features"):
    features = create_meta_features(row, go_depths, go_specificity, go_frequency_norm)
    features['pred_key'] = row['pred_key']
    features['protein_id'] = row['protein_id']
    features['go_term'] = row['go_term']
    meta_features_list.append(features)

meta_df = pd.DataFrame(meta_features_list)

print(f"[✓] Meta-features created: {meta_df.shape}")
print(f"Features: {[col for col in meta_df.columns if col not in ['pred_key', 'protein_id', 'go_term']]}")



[Creating Meta-Features Dataset]
Intersection: 4310992 predictions
Creating meta-features for 4310992 predictions...


Meta-features:   0%|          | 0/4310992 [00:00<?, ?it/s]

[✓] Meta-features created: (4310992, 17)
Features: ['score_A', 'score_B', 'score_mean', 'score_max', 'score_min', 'score_diff', 'score_std', 'score_product', 'score_ratio', 'go_depth', 'go_specificity', 'go_frequency', 'models_agree', 'both_high_conf']


$$ $$

In [11]:
print(f"\n[Preparing Meta-Learner Training Data]")

# Strategy: Use ALL predictions (not just intersection) and label with ground truth
# This ensures we have positive samples

# Merge A and B on pred_key (outer join to keep all)
print("Merging all predictions from A and B...")
all_A = A[['pred_key', 'protein_id', 'go_term', 'score']].copy()
all_A.columns = ['pred_key', 'protein_id', 'go_term', 'score_A']

all_B = B[['pred_key', 'protein_id', 'go_term', 'score']].copy()
all_B.columns = ['pred_key', 'protein_id', 'go_term', 'score_B']

# Outer merge to keep all predictions
all_merged = all_A.merge(all_B, on=['pred_key', 'protein_id', 'go_term'], how='outer')

# Fill missing scores with 0 (for predictions only in one model)
all_merged['score_A'] = all_merged['score_A'].fillna(0)
all_merged['score_B'] = all_merged['score_B'].fillna(0)

print(f"Total merged predictions: {len(all_merged)}")

# Sample for training (to save memory and time)
# We'll use a stratified sample
all_merged['label'] = all_merged['pred_key'].isin(goa_pred_keys).astype(int)

positive_count = all_merged['label'].sum()
negative_count = len(all_merged) - positive_count

print(f"Full dataset class distribution:")
print(f"  Positive: {positive_count} ({100*positive_count/len(all_merged):.2f}%)")
print(f"  Negative: {negative_count} ({100*negative_count/len(all_merged):.2f}%)")

if positive_count == 0:
    print("\n⚠️  WARNING: Still no positive samples!")
    print("This suggests predictions are completely disjoint from ground truth.")
    print("Falling back to heuristic approach...")
    
    # For intersection only (where both models have predictions)
    meta_df = all_merged[(all_merged['score_A'] > 0) & (all_merged['score_B'] > 0)].copy()
    
    # Heuristic scoring
    meta_df['meta_score'] = (
        0.4 * meta_df['score_A'] + 
        0.4 * meta_df['score_B'] + 
        0.2 * ((meta_df['score_A'] + meta_df['score_B']) / 2)
    )
    
    # Agreement bonus
    meta_df['score_diff'] = abs(meta_df['score_A'] - meta_df['score_B'])
    agreement_bonus = (1 - meta_df['score_diff']) * 0.1
    meta_df['meta_score'] = (meta_df['meta_score'] + agreement_bonus).clip(0, 1)
    
    USE_META_LEARNER = False
    
else:
    print(f"\n✓ Found {positive_count} positive samples!")
    
    # Sample for training (stratified)
    max_samples = 500000  # Limit for memory
    if len(all_merged) > max_samples:
        print(f"Sampling {max_samples} examples for training...")
        
        positives = all_merged[all_merged['label'] == 1]
        negatives = all_merged[all_merged['label'] == 0]
        
        # Keep all positives if possible, sample negatives proportionally
        if len(positives) < max_samples * 0.1:  # At least 10% positive
            n_neg = min(len(negatives), max_samples - len(positives))
            negatives_sample = negatives.sample(n=n_neg, random_state=42)
            training_df = pd.concat([positives, negatives_sample], ignore_index=True)
        else:
            training_df = all_merged.sample(n=max_samples, random_state=42)
    else:
        training_df = all_merged.copy()
    
    print(f"Training on {len(training_df)} samples")
    
    # Create meta-features for training data
    print("Creating meta-features...")
    meta_features_list = []
    for idx, row in tqdm(training_df.iterrows(), total=len(training_df), desc="Features"):
        features = create_meta_features(row, go_depths, go_specificity, go_frequency_norm)
        features['pred_key'] = row['pred_key']
        features['protein_id'] = row['protein_id']
        features['go_term'] = row['go_term']
        features['label'] = row['label']
        meta_features_list.append(features)
    
    meta_df = pd.DataFrame(meta_features_list)
    
    # Split features and labels
    feature_cols = [col for col in meta_df.columns if col not in ['pred_key', 'protein_id', 'go_term', 'label', 'meta_score']]
    
    X = meta_df[feature_cols].values
    y = meta_df['label'].values
    
    print(f"Training data shape: X={X.shape}, y={y.shape}")
    print(f"Final positive: {y.sum()} ({100*y.sum()/len(y):.2f}%)")
    print(f"Final negative: {len(y) - y.sum()} ({100*(len(y)-y.sum())/len(y):.2f}%)")
    
    USE_META_LEARNER = True


[Preparing Meta-Learner Training Data]
Merging all predictions from A and B...
Total merged predictions: 54760695
Full dataset class distribution:
  Positive: 0 (0.00%)
  Negative: 54760695 (100.00%)

⚠️  WARNING: Still no positive samples!
This suggests predictions are completely disjoint from ground truth.
Falling back to heuristic approach...


In [12]:

if USE_META_LEARNER:
    print(f"\n[Training Meta-Learner]")
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Train logistic regression
    print("Training Logistic Regression...")
    meta_model = LogisticRegression(
        max_iter=1000,
        class_weight='balanced',  # Handle class imbalance
        C=1.0,
        random_state=42,
        n_jobs=-1
    )
    
    meta_model.fit(X_scaled, y)
    
    print(f"[✓] Meta-model trained!")
    print(f"Model coefficients shape: {meta_model.coef_.shape}")
    
    # Show feature importance
    feature_importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': abs(meta_model.coef_[0])
    }).sort_values('importance', ascending=False)
    
    print(f"\nTop 10 Most Important Features:")
    print(feature_importance.head(10))
    
else:
    print(f"\n[Skipping Meta-Learner Training]")
    print("Using heuristic-based approach (already computed in Cell 11)")


[Skipping Meta-Learner Training]
Using heuristic-based approach (already computed in Cell 11)


$$ $$

In [13]:

print(f"\n[Generating Final Scores]")

if USE_META_LEARNER:
    print("Predicting with trained meta-learner...")
    
    # Need to create features for ALL predictions (not just training sample)
    print("Creating features for all predictions...")
    
    # Get intersection where both models have predictions
    intersection_df = all_merged[(all_merged['score_A'] > 0) & (all_merged['score_B'] > 0)].copy()
    
    # Create features
    pred_features_list = []
    for idx, row in tqdm(intersection_df.iterrows(), total=len(intersection_df), desc="Prediction features"):
        features = create_meta_features(row, go_depths, go_specificity, go_frequency_norm)
        features['pred_key'] = row['pred_key']
        features['protein_id'] = row['protein_id']
        features['go_term'] = row['go_term']
        pred_features_list.append(features)
    
    pred_df = pd.DataFrame(pred_features_list)
    
    # Extract features
    X_pred = pred_df[feature_cols].values
    X_pred_scaled = scaler.transform(X_pred)
    
    # Predict
    pred_df['meta_score'] = meta_model.predict_proba(X_pred_scaled)[:, 1]
    
    meta_predictions = pred_df[['protein_id', 'go_term', 'meta_score']].copy()
    meta_predictions.columns = ['protein_id', 'go_term', 'score']
    
    print(f"Meta-learner predictions: {len(meta_predictions)}")
    
else:
    print("Using heuristic scores...")
    meta_predictions = meta_df[['protein_id', 'go_term', 'meta_score']].copy()
    meta_predictions.columns = ['protein_id', 'go_term', 'score']

meta_predictions['pred_key'] = meta_predictions['protein_id'].astype(str) + '_' + meta_predictions['go_term'].astype(str)

print(f"Final meta-predictions shape: {meta_predictions.shape}")
print(f"Score distribution:\n{meta_predictions['score'].describe()}")


[Generating Final Scores]
Using heuristic scores...
Final meta-predictions shape: (4310992, 4)
Score distribution:
count    4.310992e+06
mean     2.503019e-01
std      1.559044e-01
min      1.015520e-01
25%      1.422800e-01
50%      1.826400e-01
75%      3.090400e-01
max      1.000000e+00
Name: score, dtype: float64


In [14]:
print(f"\n[Adding Non-Overlapping Predictions]")

# Predictions only in A
AnotB = A[~A.pred_key.isin(B_keys)].copy()
AnotB = AnotB[['protein_id', 'go_term', 'score', 'pred_key']]

# Predictions only in B
BnotA = B[~B.pred_key.isin(A_keys)].copy()
BnotA = BnotA[['protein_id', 'go_term', 'score', 'pred_key']]

print(f"A only: {len(AnotB)}")
print(f"B only: {len(BnotA)}")


[Adding Non-Overlapping Predictions]
A only: 8091404
B only: 42358299


In [15]:
def positive_propagation(df, parents_map, threshold=0.85, decay_factor=0.92):
    """Propagate high-confidence predictions to ancestors"""
    print(f"[Positive Propagation] Starting with {len(df)} predictions")
    print(f"[Positive Propagation] Threshold: {threshold}, Decay: {decay_factor}")
    
    high_conf = df[df['score'] >= threshold].copy()
    print(f"[Positive Propagation] High-confidence: {len(high_conf)}")
    
    new_predictions = []
    existing_keys = set(df['pred_key'])
    
    for idx, row in tqdm(high_conf.iterrows(), total=len(high_conf), desc="Propagating"):
        ancestors = get_ancestors(row['go_term'], parents_map)
        
        for ancestor in ancestors:
            pred_key = f"{row['protein_id']}_{ancestor}"
            
            if pred_key not in existing_keys:
                new_predictions.append({
                    'protein_id': row['protein_id'],
                    'go_term': ancestor,
                    'score': round(row['score'] * decay_factor, 3),
                    'pred_key': pred_key
                })
                existing_keys.add(pred_key)
    
    print(f"[Positive Propagation] Added {len(new_predictions)} ancestors")
    
    if new_predictions:
        new_df = pd.DataFrame(new_predictions)
        result = pd.concat([df, new_df], ignore_index=True)
        return result
    return df


In [16]:
print(f"\n[Combining All Predictions]")

# Combine: Ground truth + Meta predictions + A-only + B-only
submission = pd.concat([
    go_annotations[['protein_id', 'go_term', 'score', 'pred_key']],
    meta_predictions,
    AnotB,
    BnotA
], axis=0, ignore_index=True)

print(f"Before deduplication: {submission.shape}")

# Remove duplicates (keep first = ground truth priority)
submission = submission.drop_duplicates(subset=['protein_id', 'go_term'], keep='first')

print(f"After deduplication: {submission.shape}")



[Combining All Predictions]
Before deduplication: (57223180, 4)
After deduplication: (57223180, 4)


In [17]:
print(f"\n[Applying Positive Propagation]")
submission = positive_propagation(
    submission,
    parents_map,
    threshold=0.85,
    decay_factor=0.92
)

# Final cleanup
submission.drop(columns=['pred_key'], inplace=True, errors='ignore')
submission = submission.drop_duplicates(subset=['protein_id', 'go_term'], keep='first')

print(f"\n{'='*80}")
print(f"FINAL SUBMISSION SHAPE: {submission.shape}")
print(f"{'='*80}")

# ============================================================================
# CELL 18: Save Submission
# ============================================================================

print(f'\n[Saving Submission]')
submission.to_csv('submission.tsv', sep='\t', index=False, header=None)
print(f"[✓] Saved to submission.tsv")


print(f"\n[Final Statistics]")
print(f"\nScore Distribution:")
print(submission['score'].describe())

print(f"\nScore Quartiles:")
for q in [0.25, 0.50, 0.75, 0.90, 0.95, 0.99]:
    val = submission['score'].quantile(q)
    count = len(submission[submission['score'] >= val])
    print(f"  >= {val:.3f} (Q{q*100:.0f}): {count} predictions")

print(f"\n[Preview Top 20 Predictions]")
!head -20 submission.tsv

print(f"\n[✓✓✓ Pipeline Complete! ✓✓✓]")


[Applying Positive Propagation]
[Positive Propagation] Starting with 57223180 predictions
[Positive Propagation] Threshold: 0.85, Decay: 0.92
[Positive Propagation] High-confidence: 2569265


Propagating:   0%|          | 0/2569265 [00:04<?, ?it/s]

[Positive Propagation] Added 5507534 ancestors

FINAL SUBMISSION SHAPE: (62730714, 3)

[Saving Submission]
[✓] Saved to submission.tsv

[Final Statistics]

Score Distribution:
count    6.273071e+07
mean     1.957961e-01
std      4.987651e-01
min      1.000000e-03
25%      1.800000e-02
50%      3.100000e-02
75%      1.698000e-01
max      2.833200e+02
Name: score, dtype: float64

Score Quartiles:
  >= 0.018 (Q25): 48832947 predictions
  >= 0.031 (Q50): 31749085 predictions
  >= 0.170 (Q75): 15683489 predictions
  >= 0.920 (Q90): 8061865 predictions
  >= 0.920 (Q95): 8061865 predictions
  >= 1.000 (Q99): 2537171 predictions

[Preview Top 20 Predictions]
A1CHB5	GO:0042407	1.0
A1CHB5	GO:0061617	1.0
A1CHB5	GO:0005743	1.0
Q9ZFU7	GO:0000987	1.0
Q9ZFU7	GO:0003700	1.0
Q9ZFU7	GO:0006355	1.0
Q9ZFU7	GO:0043565	1.0
Q9ZFU7	GO:0003677	1.0
Q9ZFU7	GO:0046336	1.0
A0A2I6PJ05	GO:0004659	1.0
A0A2I6PJ05	GO:0008299	1.0
A0A2I6PJ05	GO:0016740	1.0
A0A2I6PJ05	GO:0046872	1.0
A0A2I6PJ05	GO:0004161	1.0
A0A2I6PJ05	GO